# Digit Recognizer by Neural Network

`神经网络` 训练识别手写数字体

`Accuracy:` 96%

---

Created on 2019-05-24

Update  on 2019-05-24

Author: 家丞

Github: https://github.com/Jiachengciel/Kaggle


---

## 一. 基础环境配置

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable

%matplotlib inline
sns.set(style='white', context='notebook', palette='deep')

ModuleNotFoundError: No module named 'torch'